# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU hangs when receiving UE Context Release Complete with mutated DU ID.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU hangs when receiving UE CONTEXT RELEASE COMPLETE after an attacker mutates identifiers during UE Context Modification; DU sends DL RRC MESSAGE TRANSFER immediately after release.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": [
      "UE CONTEXT RELEASE (CU initiated)",
      "UE CONTEXT RELEASE COMPLETE",
      "UE CONTEXT MODIFICATION",
      "DL RRC MESSAGE TRANSFER"
    ],
    "components_involved": ["gNB-CU", "gNB-DU"],
    "key_ids": {
      "transaction_id": "unknown",
      "cu_ue_f1ap_id": "unknown",
      "du_ue_f1ap_id": "unknown",
      "du_id": "unknown",
      "rnti": "mutated/incorrect",
      "pci": "unknown",
      "served_cell_list": []
    },
    "signals_or_timers": [
      "F1AP Transaction ID",
      "gNB-CU UE F1AP ID",
      "gNB-DU UE F1AP ID",
      "RRC-Container"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["UE CONTEXT RELEASE", "Release Complete", "ID pairing", "CU initiated"],
      "title": ["8.3.3 UE Context Release", "9.2.2.6 UE CONTEXT RELEASE COMPLETE"]
    },
    {
      "Keywords": ["DL RRC MESSAGE TRANSFER", "lookup by DU UE ID", "old DU UE ID"],
      "title": ["8.4.2 DL RRC Message Transfer", "9.2.3.2 DL RRC MESSAGE TRANSFER"]
    },
    {
      "Keywords": ["gNB-CU UE F1AP ID", "unique identifier", "association scope"],
      "title": ["9.3.1.4 gNB-CU UE F1AP ID"]
    },
    {
      "Keywords": ["gNB-DU UE F1AP ID", "unique identifier", "association scope"],
      "title": ["9.3.1.5 gNB-DU UE F1AP ID"]
    },
    {
      "Keywords": ["Transaction ID", "procedure scoping", "pairing"],
      "title": ["9.3.1.23 Transaction ID"]
    }
  ],
  "expected_behaviour": "For CU-initiated release, DU shall release resources and reply with UE CONTEXT RELEASE COMPLETE including both gNB-CU UE F1AP ID and gNB-DU UE F1AP ID. The CU should correlate the Complete to the ongoing release transaction (Transaction ID) and verify the ID pair consistency with stored CU<->DU UE mapping. DL RRC MESSAGE TRANSFER uses the DU UE ID to look up the UE context; if context is not retrievable, DU may trigger UE Context Release Request. Unknown or inconsistent UE F1AP ID pairs should be handled per cause semantics (e.g., reject/ignore) rather than hanging."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_RELEASE_COMPLETE",
      "reason": "CU-side handler for UE CONTEXT RELEASE COMPLETE; point to validate CU/DU UE ID pair and transaction scope."
    },
    {
      "path": "RRC/NR/rrc_gNB.c",
      "function_name": "rrc_CU_process_ue_context_release_complete",
      "reason": "RRC consumption of Release Complete keyed by CU UE ID; check for missing DU ID correlation."
    },
    {
      "path": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_handle_UL_RRC_MESSAGE_TRANSFER",
      "reason": "Example of strict CU DU-UE-ID consistency check; serves as reference for expected validation."
    },
    {
      "path": "F1AP/f1ap_du_rrc_message_transfer.c",
      "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER",
      "reason": "DU-side DL RRC path entry; forwards to MAC for UE lookup by DU UE ID."
    },
    {
      "path": "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "function_name": "dl_rrc_message_transfer",
      "reason": "DU MAC uses gNB-DU UE ID (RNTI) to find UE; establishes CU<->DU mapping if absent."
    },
    {
      "path": "F1AP/f1ap_ids.c",
      "function_name": "cu_get_f1_ue_data / cu_exists_f1_ue_data / cu_update_f1_ue_data",
      "reason": "CU mapping store for CU UE ID -> DU UE ID and assoc; used to verify ID binding."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "8.3.3.2",
      "text": "Upon reception of the UE CONTEXT RELEASE COMMAND, the gNB-DU shall release all related signalling and user data transport resources and reply with the UE CONTEXT RELEASE COMPLETE message. If included, old gNB-DU UE F1AP ID shall also be released."
    },
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "9.2.2.6",
      "text": "UE CONTEXT RELEASE COMPLETE: Direction gNB-DU -> gNB-CU. Mandatory IEs: Message Type, gNB-CU UE F1AP ID, gNB-DU UE F1AP ID. Optional: Criticality Diagnostics."
    },
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "8.4.2.2",
      "text": "DL RRC MESSAGE TRANSFER: If a UE-associated logical F1-connection exists, the message shall contain the gNB-DU UE F1AP ID IE, which should be used by gNB-DU to lookup the stored UE context. The message may include old gNB-DU UE F1AP ID for reestablishment."
    },
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "9.3.1.4 / 9.3.1.5 / 9.3.1.23",
      "text": "gNB-CU UE F1AP ID and gNB-DU UE F1AP ID uniquely identify the UE association within CU and DU respectively. Transaction ID uniquely identifies a procedure among parallel procedures; messages of the same procedure use the same Transaction ID."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_cu_ue_context_management.c",
      "location": "123-137",
      "text": "int CU_handle_UE_CONTEXT_RELEASE_COMPLETE(...){ f1ap_ue_context_rel_cplt_t cplt = {0}; if (!decode_ue_context_rel_cplt(pdu, &cplt)) { ... return -1; } MessageDef *msg_p = itti_alloc_new_message(TASK_DU_F1, 0, F1AP_UE_CONTEXT_RELEASE_COMPLETE); msg_p->ittiMsgHeader.originInstance = assoc_id; F1AP_UE_CONTEXT_RELEASE_COMPLETE(msg_p) = cplt; itti_send_msg_to_task(TASK_RRC_GNB, instance, msg_p); return 0; }"
    },
    {
      "kind": "code",
      "source": "RRC/NR/rrc_gNB.c",
      "location": "2424-2442",
      "text": "static void rrc_CU_process_ue_context_release_complete(MessageDef *msg_p){ f1ap_ue_context_rel_cplt_t *complete = &F1AP_UE_CONTEXT_RELEASE_COMPLETE(msg_p); rrc_gNB_ue_context_t *ue_context_p = rrc_gNB_get_ue_context(rrc, complete->gNB_CU_ue_id); if (!ue_context_p) { ... return; } if (UE->an_release) { rrc_gNB_send_NGAP_UE_CONTEXT_RELEASE_COMPLETE(...); rrc_remove_ue(...); } }"
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "location": "114-127",
      "text": "if (!cu_exists_f1_ue_data(msg.gNB_CU_ue_id)) { ... return -1; } f1_ue_data_t ue_data = cu_get_f1_ue_data(msg.gNB_CU_ue_id); if (ue_data.secondary_ue != msg.gNB_DU_ue_id) { LOG_E(F1AP, \"unexpected DU UE ID %u received, expected %u\" ...); free_ul_rrc_message_transfer(&msg); return -1; }"
    },
    {
      "kind": "code",
      "source": "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "location": "918-944",
      "text": "void dl_rrc_message_transfer(const f1ap_dl_rrc_message_t *dl_rrc){ ... find_nr_UE(..., dl_rrc->gNB_DU_ue_id); if (UE == NULL) { LOG_E(MAC, \"unknown UE ... ignoring\"); return; } if (!du_exists_f1_ue_data(dl_rrc->gNB_DU_ue_id)) { f1_ue_data_t new_ue_data = {.secondary_ue = dl_rrc->gNB_CU_ue_id}; du_add_f1_ue_data(...); } }"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT RELEASE COMMAND", "precond": "UE context active at DU", "postcond": "DU starts release"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT RELEASE COMPLETE (CU UE ID + DU UE ID)", "precond": "Resources released at DU", "postcond": "CU correlates IDs + transaction"},
    {"from": "Attacker", "to": "gNB-CU", "message": "Mutated UE CONTEXT RELEASE COMPLETE (wrong DU UE ID)", "precond": "On-path attacker", "postcond": "CU should reject/ignore"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "DL RRC MESSAGE TRANSFER", "precond": "UE release event or reestablishment handling", "postcond": "DU uses DU UE ID to lookup context"},
    {"from": "gNB-CU", "to": "NGAP", "message": "UE CONTEXT RELEASE COMPLETE (towards AMF)", "precond": "IDs verified, release confirmed", "postcond": "UE context removed at CU"}
  ],
  "state_machine": {
    "states": [
      "Active",
      "ReleaseCommandSent",
      "AwaitingReleaseComplete",
      "ReleaseCompleteReceived",
      "VerificationFailed",
      "Released"
    ],
    "transitions": [
      {"from": "Active", "to": "ReleaseCommandSent", "on": "Send UE CONTEXT RELEASE COMMAND"},
      {"from": "ReleaseCommandSent", "to": "AwaitingReleaseComplete", "on": "Transmit over SCTP"},
      {"from": "AwaitingReleaseComplete", "to": "ReleaseCompleteReceived", "on": "Recv UE CONTEXT RELEASE COMPLETE", "guard": "IDs consistent AND transaction matches"},
      {"from": "AwaitingReleaseComplete", "to": "VerificationFailed", "on": "Recv UE CONTEXT RELEASE COMPLETE", "guard": "Unknown or inconsistent pair of UE F1AP ID"},
      {"from": "ReleaseCompleteReceived", "to": "Released", "on": "Notify NGAP + remove UE context"},
      {"from": "VerificationFailed", "to": "Active", "on": "Ignore/notify error; keep context or retry"}
    ]
  },
  "assumptions": [
    "Attacker can mutate gNB-DU UE F1AP ID in transit.",
    "CU persists CU<->DU UE mapping and DU association for verification.",
    "CU correlates messages to Transaction ID when provided.",
    "DL RRC MESSAGE TRANSFER uses DU UE ID to find UE context at DU."
  ],
  "questions": [
    "Should CU always check DU UE ID and DU association on Release Complete?",
    "Exact error handling at CU for inconsistent ID pair: ignore vs Error Indication?",
    "Is Transaction ID carried/used in current OAI path for this procedure?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "Directionality and pairing of UE Context Release",
      "result": "PASS",
      "evidence": [
        "Spec 8.3.3.2: DU shall reply with UE CONTEXT RELEASE COMPLETE",
        "Spec 9.2.2.6: Complete includes CU/DU UE F1AP IDs"
      ]
    },
    {
      "name": "CU verifies gNB-DU UE F1AP ID against mapping on Release Complete",
      "result": "FAIL",
      "evidence": [
        "F1AP/f1ap_cu_ue_context_management.c:123-137 (no DU UE ID check before forwarding)",
        "RRC/NR/rrc_gNB.c:2424-2442 (uses only CU UE ID)",
        "Contrast: F1AP/f1ap_cu_rrc_message_transfer.c:114-127 checks DU UE ID",
        "Spec 9.3.1.5/9.3.1.4: IDs uniquely identify associations"
      ]
    },
    {
      "name": "Transaction scoping used for procedure correlation",
      "result": "UNKNOWN",
      "evidence": [
        "Spec 9.3.1.23 Transaction ID defines scoping",
        "No explicit Transaction ID handling observed in CU Release Complete path"
      ]
    },
    {
      "name": "DU lookup for DL RRC MESSAGE TRANSFER by DU UE ID",
      "result": "PASS",
      "evidence": [
        "Spec 8.4.2.2: DU uses DU UE ID to lookup context",
        "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:918-944 (find UE by gNB_DU_ue_id)"
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "CU does not validate the DU UE F1AP ID on UE CONTEXT RELEASE COMPLETE, while spec frames IDs as unique association identifiers and requires consistent pairing across procedures. This gap enables a mutated DU UE ID to bypass checks and cause a hang rather than a handled error."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "3GPP TS 38.473 §8.3.3.2",
      "3GPP TS 38.473 §9.2.2.6",
      "3GPP TS 38.473 §8.4.2.2",
      "3GPP TS 38.473 §9.3.1.4",
      "3GPP TS 38.473 §9.3.1.5",
      "3GPP TS 38.473 §9.3.1.23"
    ],
    "code_citations": [
      "F1AP/f1ap_cu_ue_context_management.c:123-137",
      "RRC/NR/rrc_gNB.c:2424-2442",
      "F1AP/f1ap_cu_rrc_message_transfer.c:114-127",
      "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:918-944"
    ],
    "expected_vs_observed": "Expected: CU correlates UE CONTEXT RELEASE COMPLETE to the ongoing release using Transaction ID and verifies the CU/DU UE F1AP ID pair against stored mapping; on mismatch, it rejects/ignores and signals error without hanging. Observed: CU handler forwards Release Complete to RRC keyed only by CU UE ID without validating DU UE ID or DU association; a mutated DU UE ID can bypass checks and lead to a hang."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "CU forwards UE CONTEXT RELEASE COMPLETE without validating the DU UE ID against the stored CU↔DU mapping, contrary to the spec’s uniqueness and pairing semantics for UE F1AP IDs. This allows a mutated DU UE ID to bypass checks and can lead to a hang instead of a handled error.",
    "citations": {
      "spec": [
        "3GPP TS 38.473 §8.3.3.2",
        "3GPP TS 38.473 §9.2.2.6",
        "3GPP TS 38.473 §8.4.2.2",
        "3GPP TS 38.473 §9.3.1.4",
        "3GPP TS 38.473 §9.3.1.5",
        "3GPP TS 38.473 §9.3.1.23"
      ],
      "code": [
        "F1AP/f1ap_cu_ue_context_management.c:123-137",
        "RRC/NR/rrc_gNB.c:2424-2443",
        "F1AP/f1ap_cu_rrc_message_transfer.c:114-128",
        "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:929-944"
      ]
    },
    "suggested_repro": [
      "Start OAI gNB-CU/DU with a UE connected.",
      "Trigger a CU-initiated UE CONTEXT RELEASE COMMAND.",
      "Intercept the returning UE CONTEXT RELEASE COMPLETE and mutate the gNB-DU UE F1AP ID.",
      "Observe CU behavior: RRC side keyed by CU UE ID proceeds without DU ID validation and may hang before NGAP UE CONTEXT RELEASE COMPLETE.",
      "Optionally send a DL RRC MESSAGE TRANSFER from DU using the (real) DU UE ID and observe DU-side context lookup."
    ],
    "risks": [
      "DoS via crafted Release Complete causing stuck UE state at CU.",
      "Context mismatch and potential resource leakage.",
      "Spec conformance risk for UE ID binding across procedures."
    ],
    "next_steps": [
      "Add DU UE ID and association check in CU_handle_UE_CONTEXT_RELEASE_COMPLETE; verify against cu_get_f1_ue_data.",
      "On mismatch, drop and send Error Indication with cause 'Unknown or inconsistent pair of UE F1AP ID'.",
      "Add Transaction ID correlation when available.",
      "Create unit/integration tests for mutated DU UE ID on Release Complete.",
      "Improve logging for ID-pair verification and errors."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.